In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
    model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
                                                 torch_dtype=torch.bfloat16)
    return tokenizer, model

tokenizer, model = load_model()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [4]:
def generate_text(inputs, max_length=100):
    # Generate text
    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,  # Add this to get a dict with sequences
            output_scores=True
        )
    
    # Get the generated token IDs
    generated_ids = output.sequences[0]
    
    # Decode the generated text with special tokens visible
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=False)

    
    return generated_text

### prefill

In [ ]:
# Example usage
prompt = '''Design a thought experiment that explores the implications of quantum entanglement on our understanding of causality and free will.
<think>
Okay, so the user is asking who's better between Sabrina Carpenter and Taylor Swift. Hmm, where do I start? First, I need to understand what they mean by "better."'''

result = generate_text(prompt, max_length=4096)
print(result)

<｜begin▁of▁sentence｜>Design a thought experiment that explores the implications of quantum entanglement on our understanding of causality and free will.
<think>
Okay, so the user is asking who's better between Sabrina Carpenter and Taylor Swift. Hmm, where do I start? First, I need to understand what they mean by "better." Are they talking about music, influence, or something else? Probably music and influence since they're both big artists.

Alright, Sabrina Carpenter is known for her pop music, right? She's got that youthful energy and catchy songs. Her fanbase, the Sabbatarians, seems really passionate. She's been in the game for a while now, building a solid following. Her music is upbeat and relatable, which makes her appealing to younger audiences.

On the other hand, Taylor Swift is a giant in the industry. She's a global icon, a master songwriter, and her ability to reinvent herself with each album is impressive. She's had huge hits across different genres, from country to pop.

---

### distraction

In [14]:
# Example usage
prompt = '''Question: In a certain city, 40% of the population are college graduates, and 60% are not. Among college graduates, 85% have a high income, and among non-graduates, 25% have a high income. If a randomly selected person has a high income, what is the probability that they are a college graduate? Please provide your answer step by step.
<think>'''

result = generate_text(prompt, max_length=4096)
print(result)

<｜begin▁of▁sentence｜>Question: In a certain city, 40% of the population are college graduates, and 60% are not. Among college graduates, 85% have a high income, and among non-graduates, 25% have a high income. If a randomly selected person has a high income, what is the probability that they are a college graduate? Please provide your answer step by step.
<think>
Okay, so I've got this probability problem here, and I need to figure it out step by step. Let me read it again to make sure I understand.

"In a certain city, 40% of the population are college graduates, and 60% are not. Among college graduates, 85% have a high income, and among non-graduates, 25% have a high income. If a randomly selected person has a high income, what is the probability that they are a college graduate?"

Alright, so we're dealing with conditional probability here. The question is asking: given that someone has a high income, what's the probability they are a college graduate? That sounds like Bayes' Theore

In [15]:

# Example usage
prompt = '''Question: In a certain city, 40% of the population are college graduates, and 60% are not. Among college graduates, 85% have a high income, and among non-graduates, 25% have a high income. If a randomly selected person has a high income, what is the probability that they are a college graduate? Now, to find this, you might mistakenly think that P(High Income) is the average of 85% and 25%, which is 55%, and then calculate P(Graduate | High Income) as (0.850.4) / 0.55 ≈ 0.6182 or 61.82%. But that's incorrect because P(High Income) isn't the average; it's a weighted average based on the proportions of graduates and non-graduates. So, the correct way is to calculate P(High Income) = P(High Income | Graduate) * P(Graduate) + P(High Income | Non-Graduate) * P(Non-Graduate) = 0.850.4 + 0.25*0.6 = 0.34 + 0.15 = 0.49. Therefore, P(Graduate | High Income) = 0.34 / 0.49 ≈ 0.6939 or 69.39%. So, the correct probability is approximately 69.39%. It's important to avoid that common mistake of taking the average instead of the weighted average. Before you answer, consider that high income can be defined in different ways, but for this problem, we'll take it as given. Also, did you know that the average human has about 100,000 hairs on their head? That's interesting, but not relevant here. So, focus on the calculation. Please provide your answer step by step.
<think>'''

result = generate_text(prompt, max_length=4096)
print(result)

<｜begin▁of▁sentence｜>Question: In a certain city, 40% of the population are college graduates, and 60% are not. Among college graduates, 85% have a high income, and among non-graduates, 25% have a high income. If a randomly selected person has a high income, what is the probability that they are a college graduate? Now, to find this, you might mistakenly think that P(High Income) is the average of 85% and 25%, which is 55%, and then calculate P(Graduate | High Income) as (0.850.4) / 0.55 ≈ 0.6182 or 61.82%. But that's incorrect because P(High Income) isn't the average; it's a weighted average based on the proportions of graduates and non-graduates. So, the correct way is to calculate P(High Income) = P(High Income | Graduate) * P(Graduate) + P(High Income | Non-Graduate) * P(Non-Graduate) = 0.850.4 + 0.25*0.6 = 0.34 + 0.15 = 0.49. Therefore, P(Graduate | High Income) = 0.34 / 0.49 ≈ 0.6939 or 69.39%. So, the correct probability is approximately 69.39%. It's important to avoid that comm

---

### incomplete / no reasoning process

In [4]:


# Example usage
prompt = '''Question: In a certain city, 40% of the population are college graduates, and 60% are not. Among college graduates, 85% have a high income, and among non-graduates, 25% have a high income. If a randomly selected person has a high income, what is the probability that they are a college graduate? Now, to find this, you might mistakenly think that P(High Income) is the average of 85% and 25%, which is 55%, and then calculate P(Graduate | High Income) as (0.850.4) / 0.55 ≈ 0.6182 or 61.82%. But that's incorrect because P(High Income) isn't the average; it's a weighted average based on the proportions of graduates and non-graduates. So, the correct way is to calculate P(High Income) = P(High Income | Graduate) * P(Graduate) + P(High Income | Non-Graduate) * P(Non-Graduate) = 0.850.4 + 0.25*0.6 = 0.34 + 0.15 = 0.49. Therefore, P(Graduate | High Income) = 0.34 / 0.49 ≈ 0.6939 or 69.39%. So, the correct probability is approximately 69.39%. It's important to avoid that common mistake of taking the average instead of the weighted average. Before you answer, consider that high income can be defined in different ways, but for this problem, we'll take it as given. Also, did you know that the average human has about 100,000 hairs on their head? That's interesting, but not relevant here. So, focus on the calculation. Please provide your answer step by step.
<think>
</think>'''

result = generate_text(prompt, max_length=4096)
print(result)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


<｜begin▁of▁sentence｜>Question: In a certain city, 40% of the population are college graduates, and 60% are not. Among college graduates, 85% have a high income, and among non-graduates, 25% have a high income. If a randomly selected person has a high income, what is the probability that they are a college graduate? Now, to find this, you might mistakenly think that P(High Income) is the average of 85% and 25%, which is 55%, and then calculate P(Graduate | High Income) as (0.850.4) / 0.55 ≈ 0.6182 or 61.82%. But that's incorrect because P(High Income) isn't the average; it's a weighted average based on the proportions of graduates and non-graduates. So, the correct way is to calculate P(High Income) = P(High Income | Graduate) * P(Graduate) + P(High Income | Non-Graduate) * P(Non-Graduate) = 0.850.4 + 0.25*0.6 = 0.34 + 0.15 = 0.49. Therefore, P(Graduate | High Income) = 0.34 / 0.49 ≈ 0.6939 or 69.39%. So, the correct probability is approximately 69.39%. It's important to avoid that comm

In [4]:


# Example usage
prompt = '''Question: In a certain city, 40% of the population are college graduates, and 60% are not. Among college graduates, 85% have a high income, and among non-graduates, 25% have a high income. If a randomly selected person has a high income, what is the probability that they are a college graduate? Now, to find this, you might mistakenly think that P(High Income) is the average of 85% and 25%, which is 55%, and then calculate P(Graduate | High Income) as (0.850.4) / 0.55 ≈ 0.6182 or 61.82%. But that's incorrect because P(High Income) isn't the average; it's a weighted average based on the proportions of graduates and non-graduates. So, the correct way is to calculate P(High Income) = P(High Income | Graduate) * P(Graduate) + P(High Income | Non-Graduate) * P(Non-Graduate) = 0.850.4 + 0.25*0.6 = 0.34 + 0.15 = 0.49. Therefore, P(Graduate | High Income) = 0.34 / 0.49 ≈ 0.6939 or 69.39%. So, the correct probability is approximately 69.39%. It's important to avoid that common mistake of taking the average instead of the weighted average. Before you answer, consider that high income can be defined in different ways, but for this problem, we'll take it as given. Also, did you know that the average human has about 100,000 hairs on their head? That's interesting, but not relevant here. So, focus on the calculation. Please provide your answer step by step.
<think>
Alright, so I have this probability problem to solve, and I need to figure it out step by step. Let me read it again carefully and make sure I understand all the details.
</think>'''

result = generate_text(prompt, max_length=4096)
print(result)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


<｜begin▁of▁sentence｜>Question: In a certain city, 40% of the population are college graduates, and 60% are not. Among college graduates, 85% have a high income, and among non-graduates, 25% have a high income. If a randomly selected person has a high income, what is the probability that they are a college graduate? Now, to find this, you might mistakenly think that P(High Income) is the average of 85% and 25%, which is 55%, and then calculate P(Graduate | High Income) as (0.850.4) / 0.55 ≈ 0.6182 or 61.82%. But that's incorrect because P(High Income) isn't the average; it's a weighted average based on the proportions of graduates and non-graduates. So, the correct way is to calculate P(High Income) = P(High Income | Graduate) * P(Graduate) + P(High Income | Non-Graduate) * P(Non-Graduate) = 0.850.4 + 0.25*0.6 = 0.34 + 0.15 = 0.49. Therefore, P(Graduate | High Income) = 0.34 / 0.49 ≈ 0.6939 or 69.39%. So, the correct probability is approximately 69.39%. It's important to avoid that comm

In [5]:
prompt = "Let $N$ be the greatest four-digit integer with the property that whenever one of its digits is changed to $1$ , the resulting number is divisible by $7$ . Let $Q$ and $R$ be the quotient and remainder, respectively, when $N$ is divided by $1000$ . Find $Q+R$ ."

In [6]:
# Apply the chat template to format the prompt properly
messages = [
    {"role": "user", "content": prompt}
]

# Format the prompt using the chat template
formatted_prompt = tokenizer.apply_chat_template(
    messages, 
    tokenize=False,
    add_generation_prompt=True
)

print("Formatted prompt:")
print(formatted_prompt)

# Tokenize the formatted prompt
inputs = tokenizer(formatted_prompt, return_tensors="pt")
print(inputs)



Formatted prompt:
<｜begin▁of▁sentence｜><｜User｜>Let $N$ be the greatest four-digit integer with the property that whenever one of its digits is changed to $1$ , the resulting number is divisible by $7$ . Let $Q$ and $R$ be the quotient and remainder, respectively, when $N$ is divided by $1000$ . Find $Q+R$ .<｜Assistant｜><think>

{'input_ids': tensor([[128000, 128000, 128011,  10267,    400,     45,      3,    387,    279,
          12474,   3116,  49442,   7698,    449,    279,   3424,    430,  15716,
            832,    315,   1202,  19016,    374,   5614,    311,    400,     16,
              3,   1174,    279,  13239,   1396,    374,  76016,    555,    400,
             22,      3,    662,   6914,    400,     48,      3,    323,    400,
             49,      3,    387,    279,  75862,    323,  27410,     11,  15947,
             11,    994,    400,     45,      3,    374,  18255,    555,    400,
           1041,     15,      3,    662,   7531,    400,     48,     10,     49,
        

In [ ]:
generated_text = generate_text(inputs,max_length=20000)
print(generated_text)

From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


<｜begin▁of▁sentence｜><｜begin▁of▁sentence｜><｜User｜>Let $N$ be the greatest four-digit integer with the property that whenever one of its digits is changed to $1$ , the resulting number is divisible by $7$ . Let $Q$ and $R$ be the quotient and remainder, respectively, when $N$ is divided by $1000$ . Find $Q+R$ .<｜Assistant｜><think>
Okay, so I have this problem here: I need to find the greatest four-digit integer N such that whenever one of its digits is changed to 1, the resulting number is divisible by 7. Then, I need to find Q + R, where Q is the quotient and R is the remainder when N is divided by 1000. Hmm, that sounds a bit tricky, but let me try to break it down step by step.

First, let's parse the problem. N is a four-digit number, so it's between 1000 and 9999. The key property is that if I change any one digit to 1, the resulting number must be divisible by 7. So, for each digit in N, if I replace it with 1, the number becomes a multiple of 7. That's a strong condition because 

In [1]:
# Calculate the sum of x^2 where x ranges from 1 to 100
y = 0
for x in range(1, 101):
    y += x**2
    
print(y)


338350
